In [6]:
import pandas as pd

df = pd.read_csv("data/geral_2024.csv").sort_values("Total com corte", ascending=False).reset_index(drop=True)
df["Classificação"] = df.index + 1
df

,Classificação,Players,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Total,Total com corte
0,1,Carlos Magno,15.0,0.0,17.0,38.5,26.5,38.0,14.0,36.0,40.0,NaN,NaN,NaN,NaN,211.0
1,2,Perroni,29.5,37.5,0.0,23.5,0.0,30.5,28.0,12.0,26.0,NaN,NaN,NaN,NaN,187.0
2,3,Gleisson,26.0,29.0,28.0,10.0,22.5,21.0,12.5,21.5,20.0,NaN,NaN,NaN,NaN,170.5
3,4,Nicin,18.0,22.0,22.5,28.5,23.0,23.5,25.5,22.0,18.5,NaN,NaN,NaN,NaN,167.0
4,5,Diu,12.0,18.0,25.5,26.5,14.0,18.0,19.0,16.0,28.0,NaN,NaN,NaN,NaN,151.0
5,6,Guilherme,21.5,11.0,0.0,19.0,28.5,17.0,15.0,29.0,14.5,NaN,NaN,NaN,NaN,144.5
6,7,Picanha,23.0,18.5,17.0,13.5,20.5,11.0,37.0,0.0,13.0,NaN,NaN,NaN,NaN,142.5
7,8,Ricardinho,0.0,20.0,0.0,14.5,36.0,27.0,12.0,12.5,16.5,NaN,NaN,NaN,NaN,138.5
8,9,Tantan,17.0,12.5,18.0,17.5,13.0,11.0,22.5,19.0,22.5,NaN,NaN,NaN,NaN,127.0
9,10,Valdez,0.0,26.5,36.5,12.5,16.0,0.0,17.0,14.0,NaN,NaN,NaN,NaN,NaN,122.5


In [1]:
from datetime import datetime
import pandas as pd
import streamlit as st

def update_table_geral(month):
    current_date = datetime.now()
    geral_filename = f"data/geral_{current_date.year}.csv"
    monthly_filename = f"data/classificacao_{current_date.year}_{month}.csv"

    # month = st.selectbox("Mês para atualizar a classificação geral:", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], index=current_date.month-1)
    df_month = pd.read_csv(monthly_filename).sort_values("Total", ascending=False).reset_index(drop=True)
    df = pd.read_csv(geral_filename).sort_values("Total com corte", ascending=False).reset_index(drop=True)

    cols_rodadas = [c for c in df.columns if c.startswith("Rodada")]

    def apply_total_com_corte(s):
        cleanedList = sorted([x for x in s[cols_rodadas].values.tolist() 
                                if str(x) != 'nan'], reverse=True)
        return sum([value for value in cleanedList[0:-2]])

    # Para cada novo jogador do mes
    for player in df_month.Players.values:
        # Se o jogador ja existe
        if player in df.Players.values:
            df.loc[df["Players"] == player, [f"Rodada {month}"]] = df_month[df_month["Players"] == player].Total.values[0]
        else:
            s = len(df)
            df.loc[s, "Classificação"] = s+1
            df.loc[s, "Players"] = player

            # Fill all rodadas (0 before this month)
            for rodada in [c for c in df.columns if c.startswith("Rodada")]:
                number_rodada = int(rodada.split("Rodada")[1])
                if number_rodada < month:
                    df.loc[s, rodada] = 0
                elif number_rodada == month:
                    df.loc[s, rodada] = df_month[df_month["Players"] == player].Total.values[0]
                    
    # Para cada jogador que faltou no mes:
    for player in df.Players.values:
        if player not in df_month.Players.values:
            df.loc[df["Players"] == player, [f"Rodada {month}"]] = 0

    df["Total"] = df[cols_rodadas].sum()
    df["Total com corte"] = df.apply(apply_total_com_corte, axis=1)
    st.success("Saved!")
    df.to_csv("data/geral_2024.csv", index=False)

In [2]:
df = update_table_geral("9")
df

2024-09-25 09:16:15.597 
  command:

    streamlit run /home/gcdelgado/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
